In [175]:
import csv
import os

import gspread
import imdb
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# Environment
load_dotenv()
CREDENTIALS = os.getenv('GOOGLE_CREDENTIALS')
SHEET = os.getenv('GOOGLESHEET_ID')

In [4]:
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS + '.json', scope)
client = gspread.authorize(credentials)
sh = client.open_by_key(SHEET)

In [174]:
def upload_to_sheets(sh, data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data = list(csv.reader(f))

    worksheet_name = str(Path(data_path).name).split(".")[0]
    try:
        sh.add_worksheet(title=worksheet_name, rows=str(len(csv_file)), cols=str(len(csv_file[0])))
    except:
        print(f"Worksheet {worksheet_name} already exists")

    sh.values_update(
        worksheet_name,
        params={'valueInputOption': 'USER_ENTERED'},
        body={'values': data}
    )

In [177]:
data_files = Path("data").glob("**/*")
csv_files = []
for f in data_files:
    if "csv" in str(f):
        upload_to_sheets(sh, f)

In [37]:
ia = imdb.IMDb()

In [100]:
df = pd.read_csv("data/discussions.csv")
df[df["Type"] == "watch"]

In [101]:
watch_list = list(df[df["Type"] == "watch"]["Title"])

In [147]:
data = []
data.append("Title,Year,Directors,Metascore\n")
for title in watch_list:
    candidate_info = ia.search_movie(title)[0]
    year = candidate_info["year"]
    directors = "\""
    movie_data = ia.get_movie(candidate_info.movieID)
    if "directors" in movie_data:
        for d in movie_data["directors"]:
            directors += d["name"] + ","
    directors = directors[:-1] if "," in directors else directors
    directors += "\""
    metascore = ""
    review_data = ia.get_movie_critic_reviews(candidate_info.movieID)["data"]
    if "metascore" in review_data:
        metascore = review_data["metascore"]
    data.append(f"\"{title}\",{year},{directors},{metascore}\n")

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [10:57<00:00,  6.64s/it]


In [164]:
with open(f"data/watch_info.csv", "w", encoding='utf-8') as f:
    f.writelines(data)

In [180]:
upload_to_sheets(sh, "data/watch_info.csv")

In [ ]:
    candidate_info = ia.search_movie(title)[0]
    year = candidate_info["year"]
    directors = "\""
    movie_data = ia.get_movie(candidate_info.movieID)
    if "directors" in movie_data:
        for d in movie_data["directors"]:
            directors += d["name"] + ","
    directors = directors[:-1] if "," in directors else directors
    directors += "\""
    metascore = ""
    review_data = ia.get_movie_critic_reviews(candidate_info.movieID)["data"]
    if "metascore" in review_data:
        metascore = review_data["metascore"]
    data.append(f"\"{title}\",{year},{directors},{metascore}\n")